In [2]:
import os

import joblib
import numpy as np
import open3d as o3d
import pandas as pd
import torch
import torch.nn as nn

from PointFlow.args import get_parser
from PointFlow.models.networks import PointFlow

ModuleNotFoundError: No module named 'models.flow'

In [2]:
parser = get_parser()
args = parser.parse_args("--cates all --resume_checkpoint logs/shapenet/PointFlow/all/checkpoints/model.pt --dims 512-512-512 --use_deterministic_encoder --evaluate_recon --resume_dataset_mean logs/shapenet/PointFlow/all/checkpoints/train_set_mean.npy --resume_dataset_std logs/shapenet/PointFlow/all/checkpoints/train_set_std.npy".split())

In [6]:
def load_model(args):
    model = PointFlow(args)

    def _transform_(m):
        return nn.DataParallel(m)

    model.multi_gpu_wrapper(_transform_)

    print("Resume Path:%s" % args.resume_checkpoint)
    checkpoint = torch.load(args.resume_checkpoint, map_location='cpu')
    model.load_state_dict(checkpoint)
    _ = model.eval()
    return model


def load_flow(args):
    save_dir = os.path.dirname(args.resume_checkpoint)
    flow = torch.load(os.path.join(save_dir, 'flow.pkt'))
    return flow


def load_label_encoder(args):
    save_dir = os.path.dirname(args.resume_checkpoint)
    le = joblib.load(os.path.join(save_dir, 'label_encoder.joblib'))
    return le


def load_data(args):
    save_dir = os.path.dirname(args.resume_checkpoint)
    path_df_train = os.path.join(save_dir, 'train_latent_space.csv')
    path_df_test = os.path.join(save_dir, 'test_latent_space.csv')

    df_train = pd.read_csv(path_df_train)
    df_test = pd.read_csv(path_df_test)
    return df_train, df_test

In [7]:
def visualize_cloud(points):
    pcl = o3d.geometry.PointCloud()
    pts = points.reshape(-1, 3)
    pcl.points = o3d.utility.Vector3dVector(pts)
    o3d.visualization.draw_geometries([pcl])

In [8]:
model = load_model(args)
flow = load_flow(args)
# le = load_label_encoder(args)
df_train, df_test = load_data(args)

Number of trainable parameters of Point CNF: 927513
Resume Path:pretrained_models/ae/all/checkpoint.pt


In [23]:
N = 2048*5

In [24]:
classes = ['airplane', 'bag', 'basket', 'bathtub', 'bed', 'bench',
       'birdhouse', 'bookshelf', 'bottle', 'bowl', 'bus', 'cabinet',
       'camera', 'can', 'cap', 'car', 'cellphone', 'chair', 'clock',
       'dishwasher', 'earphone', 'faucet', 'file', 'guitar', 'helmet',
       'jar', 'keyboard', 'knife', 'lamp', 'laptop', 'mailbox',
       'microphone', 'microwave', 'monitor', 'motorcycle', 'mug', 'piano',
       'pillow', 'pistol', 'pot', 'printer', 'remote_control', 'rifle',
       'rocket', 'skateboard', 'sofa', 'speaker', 'stove', 'table',
       'telephone', 'tin_can', 'tower', 'train', 'vessel', 'washer']

In [25]:
classes[35]

'mug'

In [27]:
label = 35
num_classes = 55

context = torch.zeros((1, num_classes))
context[0, label] = 1

results = model.decode(flow.sample(1, context).squeeze(0), N)

visualize_cloud(results[1].cpu().detach().numpy())